# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_file = "../output_data/cities.csv"
cities_df = pd.read_csv(cities_file)
cities_df.head()


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Katsuura,35.1333,140.3000,67.21,91,100,19.51,JP,1665610698
1,Harper,4.3750,-7.7169,77.22,82,98,7.16,LR,1665610530
2,Torbay,47.6666,-52.7314,50.76,76,75,5.75,CA,1665610426
3,Chokurdakh,70.6333,147.9167,1.04,90,56,5.01,RU,1665610574
4,Korla,41.7597,86.1469,47.48,50,0,3.29,CN,1665610699


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# set up lat and long as locations and humidity as weight
#drop NaN values
cities_df = cities_df.dropna()
locations = cities_df[["Lat", "Long"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

#add heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(humidity), point_radius = 2)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#narrow down cities to ideal weather condition
#max temp between 70 and 80, wind speed less than 10mph, zero cloudiness
#drop any NaN values
ideal_weather = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70)]
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] < 10]
ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"] ==0]
ideal_weather = ideal_weather.dropna()



ideal_weather




,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Gushikawa,26.3544,127.8686,76.23,81,0,6.91,JP,1665610701
140,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,72.45,29,0,3.24,AR,1665610742
166,Mount Isa,-20.7333,139.5000,74.97,69,0,0.00,AU,1665610751
194,Pasni,25.2631,63.4710,78.57,61,0,8.99,PK,1665610762
257,Pacific Grove,36.6177,-121.9166,73.38,83,0,9.22,US,1665610782
285,Graham,47.0529,-122.2943,71.94,50,0,6.91,US,1665610773
345,Dumas,35.8656,-101.9732,73.62,24,0,5.75,US,1665610814
351,Al Bardīyah,31.7561,25.0865,71.71,75,0,7.11,LY,1665610816
356,Macaé,-22.3708,-41.7869,74.95,88,0,9.22,BR,1665610819
362,Bhakkar,31.6253,71.0657,75.33,38,0,4.97,PK,1665610821


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create dataframe
hotel_df = ideal_weather.copy()

#store hotel name
hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Gushikawa,26.3544,127.8686,76.23,81,0,6.91,JP,1665610701,
140,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,72.45,29,0,3.24,AR,1665610742,
166,Mount Isa,-20.7333,139.5000,74.97,69,0,0.00,AU,1665610751,
194,Pasni,25.2631,63.4710,78.57,61,0,8.99,PK,1665610762,
257,Pacific Grove,36.6177,-121.9166,73.38,83,0,9.22,US,1665610782,
285,Graham,47.0529,-122.2943,71.94,50,0,6.91,US,1665610773,
345,Dumas,35.8656,-101.9732,73.62,24,0,5.75,US,1665610814,
351,Al Bardīyah,31.7561,25.0865,71.71,75,0,7.11,LY,1665610816,
356,Macaé,-22.3708,-41.7869,74.95,88,0,9.22,BR,1665610819,
362,Bhakkar,31.6253,71.0657,75.33,38,0,4.97,PK,1665610821,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "['Lng'] not in index"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
